In [4]:
!pip install ultralytics

In [5]:
## importing required libraries
import os
import shutil
import random

!pip install tqdm --upgrade
from tqdm.notebook import tqdm

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
train_path_img = "./yolo_data/images/train/"
train_path_label = "./yolo_data/labels/train/"
val_path_img = "./yolo_data/images/val/"
val_path_label = "./yolo_data/labels/val/"
test_path = "./yolo_data/test"

In [8]:
def train_test_split(path, neg_path=None, split=0.2):
    print("------ PROCESS STARTED -------")

    files = list(set([name[:-4] for name in os.listdir(path)]))  # Removing duplicate names
    print(f"--- This folder has a total number of {len(files)} images---")

    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    # Creating required directories
    os.makedirs(train_path_img, exist_ok=True)
    os.makedirs(train_path_label, exist_ok=True)
    os.makedirs(val_path_img, exist_ok=True)
    os.makedirs(val_path_label, exist_ok=True)

    # Copying images to train folder
    for filex in tqdm(files[:train_size]):
        if filex == 'classes':
            continue
        img_src = os.path.join(path, filex + '.png')
        label_src = os.path.join(path, filex + '.txt')

        if os.path.exists(img_src) and os.path.exists(label_src):
            shutil.copy2(img_src, os.path.join(train_path_img, filex + '.png'))
            shutil.copy2(label_src, os.path.join(train_path_label, filex + '.txt'))
        else:
            print(f"Warning: {filex} does not exist in the directory.")

    print(f"------ Training data created with 80% split {train_size} images -------")

    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)]))  # Removing duplicate names
        for filex in tqdm(neg_images):
            img_src = os.path.join(neg_path, filex + '.png')

            if os.path.exists(img_src):
                shutil.copy2(img_src, os.path.join(train_path_img, filex + '.png'))
            else:
                print(f"Warning: {filex} does not exist in the negative images directory.")

        print(f"------ Total {len(neg_images)} negative images added to the training data -------")
        print(f"------ TOTAL Training data created with {train_size + len(neg_images)} images -------")

    # Copying images to validation folder
    for filex in tqdm(files[train_size:]):
        if filex == 'classes':
            continue
        img_src = os.path.join(path, filex + '.png')
        label_src = os.path.join(path, filex + '.txt')

        if os.path.exists(img_src) and os.path.exists(label_src):
            shutil.copy2(img_src, os.path.join(val_path_img, filex + '.png'))
            shutil.copy2(label_src, os.path.join(val_path_label, filex + '.txt'))
        else:
            print(f"Warning: {filex} does not exist in the directory.")

    print(f"------ Testing data created with a total of {test_size} images -------")
    print("------ TASK COMPLETED -------")

# Splitting the data into train-test and creating train.txt and test.txt files
train_test_split('/content/drive/MyDrive/yololetters/data')  # Without negative images
# train_test_split('./data/', './negative_images/')  # If you want to feed negative images


------ PROCESS STARTED -------
--- This folder has a total number of 2070 images---


  0%|          | 0/1656 [00:00<?, ?it/s]

------ Training data created with 80% split 1656 images -------


  0%|          | 0/414 [00:00<?, ?it/s]

------ Testing data created with a total of 414 images -------
------ TASK COMPLETED -------


In [9]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.41 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 30.4/78.2 GB disk)


In [11]:
!yolo task=detect mode=train model=yolov8s.pt data=/content/drive/MyDrive/yololetters/data.yaml epochs=50 imgsz=640 batch=64 project=/content/drive/MyDrive/yololetters name=letters


Ultralytics YOLOv8.2.41 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/yololetters/data.yaml, epochs=50, time=None, patience=100, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/yololetters, name=letters3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_

In [ ]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/yolov8/training_results/plates2/weights/best.pt conf=0.55 source=/content/drive/MyDrive/yolov8/test_images

Ultralytics YOLOv8.2.39 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs

image 1/14 /content/drive/MyDrive/yolov8/test_images/0004_jpg.rf.0231b73b7cf7d709633c0590860c9a92.jpg: 640x640 1 plate, 16.4ms
image 2/14 /content/drive/MyDrive/yolov8/test_images/0007_jpg.rf.aac5bca9738b2e6b80b14615261b15b3.jpg: 640x640 1 plate, 16.4ms
image 3/14 /content/drive/MyDrive/yolov8/test_images/0008_jpg.rf.2a7bc7b1f00025ad1ea86b765c1e1004.jpg: 640x640 1 plate, 16.4ms
image 4/14 /content/drive/MyDrive/yolov8/test_images/0009_jpg.rf.5148452ffadb79377def82aa48e7b7ab.jpg: 640x640 1 plate, 16.4ms
image 5/14 /content/drive/MyDrive/yolov8/test_images/0012_jpg.rf.7521797aed0c16df17478fb2b52245de.jpg: 640x640 1 plate, 16.4ms
image 6/14 /content/drive/MyDrive/yolov8/test_images/0017_jpg.rf.45241f6bd0b1b54c298f5e3ba1a402e4.jpg: 640x640 1 plate, 16.5ms
image 7/14 /content/drive/MyDrive/yolov8/test_images/0025_jpg.rf.6a65

In [ ]:
!cp -r /content/runs/detect/predict /content/drive/MyDrive/yolov8/output

In [20]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/yololetters/letters3/weights/best.pt conf=0.4 source=/content/cropped_output_1.0.png

Ultralytics YOLOv8.2.41 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11136033 parameters, 0 gradients, 28.5 GFLOPs

image 1/1 /content/cropped_output_1.0.png: 384x640 2 3ens, 2 3s, 1 4, 1 9, 105.1ms
Speed: 3.1ms preprocess, 105.1ms inference, 574.5ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict7
💡 Learn more at https://docs.ultralytics.com/modes/predict


In [19]:
cp -r /content/runs/detect/predict2 /content/drive/MyDrive/yolov8/output

cp: cannot create directory '/content/drive/MyDrive/yolov8/output': No such file or directory


In [ ]:
!yolo mode=export model=/content/drive/MyDrive/yolov8/training_results/plates4/weights/best.pt format=onnx

Ultralytics YOLOv8.1.29 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs

PyTorch: starting from '/content/drive/MyDrive/yolov8/training_results/plates4/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (128.0 MB)
requirements: Ultralytics requirement ['onnx>=1.12.0'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 74.3 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 9.0s, installed 1 package: ['onnx>=1.12.0']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.15.0 opset 17...
ONNX: export success ✅ 10.6s, saved as '/content/drive/MyDrive/yolov8/training_results/plates4/weights/best.onnx' (42.6 MB)

Export complete (13.4s)
Results saved to /content/drive/MyDrive/yolov8/training_results/plates4/weights
Predict:         yolo predict task=detect mo